#from allcools page: https://lhqing.github.io/ALLCools/cell_level/step_by_step/100kb/03-HighlyVariableFeatureSelection.html
Calculate Highly Variable Features And Get mC Fraction AnnData¶

Purpose
The purpose of this step is to select highly variable features (HVF) and generate cell-by-feature methylation fraction matrix for clustering. The highly variable features are selected by comparing feature’s normalized dispersion among cells.

In [ ]:
Input¶
Filtered cell metadata;

MCDS files;

Feature list from basic feature filtering

Output
cell-by-HVF methylation fraction matrix stored in AnnData format, e.g., mCH adata and mCG adata.

In [5]:
cd /share/lasallelab/Ensi/project/allcools/oocytes/

/share/lasallelab/Ensi/project/allcools/oocytes


In [1]:
conda activate /share/lasallelab/Ensi/anaconda3/allcools/

usage: conda [-h] [--no-plugins] [-V] COMMAND ...
conda: error: argument COMMAND: invalid choice: 'activate' (choose from 'clean', 'compare', 'config', 'create', 'info', 'init', 'install', 'list', 'notices', 'package', 'remove', 'uninstall', 'rename', 'run', 'search', 'update', 'upgrade', 'build', 'content-trust', 'convert', 'debug', 'develop', 'doctor', 'index', 'inspect', 'metapackage', 'render', 'repoquery', 'skeleton', 'verify', 'token', 'repo', 'env', 'server', 'pack')


In [1]:
import pathlib
import pandas as pd
import dask
from ALLCools.mcds import MCDS

In [5]:
# If True, will load all data into memory.
# Computation will be much faster, but also very memory intensive, only use this for small number of cells (<10,000)
load = True

# change this to the path to your filtered metadata
metadata_path = '../../CellMetadata_PassQC_alloocytes_withyear_mtreads_sub.csv'

# change this to the paths to your MCDS files
mcds_path = '../mcds'

# Feature list after basic filter
feature_path = 'FeatureList.BasicFilter.txt'

# Dimension name used to do clustering
obs_dim = 'cell'  # observation
var_dim = 'chrom100k'  # feature

# HVF method:
# SVR: regression based
# Bins: normalize dispersion per bin
hvf_method = 'SVR'
mch_pattern = 'CHN'
mcg_pattern = 'CGN'
n_top_feature = 31645

# Downsample cells
downsample = 20000

In [6]:
metadata = pd.read_csv(metadata_path, index_col=0, sep="\t")
metadata = metadata[metadata["Type"] == "PO"]
total_cells = metadata.shape[0]
print(f'Metadata of {total_cells} cells')
metadata

Metadata of 46 cells


,SampleI,AnimalID,Date,Animdate,Collection,Year,Type.of.sample,WellID,WellBarcode,Type,TotalRead,mCGFrac,Group,SampleName,Path,Bamfile,Total.Reads,Mitochondrial.Reads,MT.Fraction,MT.Percentage
SampleID,,,,,,,,,,,,,,,,,,,,
4146045001PO3NC4G4C,4146045001PO3NC4G4,41460,45001,f2,2,2023,Pooled Oocytes (4) #3,G4,TCGTGCAT,PO,2200000,0.689,Control,4146045001PO3NC4G4C_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,41460_45001_PO3_NC4_G4_1_val_1_bismark_bt2_pe....,2535512,8738,NaN,NaN
4129944963PO2NC4M1S,4129944963PO2NC4M1,41299,44963,d2,1,2023,Pooled Oocytes (4) #2,M1,CACGCAAT,PO,2300000,0.684,Stressed,4129944963PO2NC4M1S_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,41299_44963_PO2_NC4_M1_1_val_1_bismark_bt2_pe....,2270808,38790,NaN,NaN
4520844650PO1NC5A7S,4520844650PO1NC5A7,45208,44650,n1,1,2022,Pooled Oocytes (5) #1,A7,ATCGTCTC,PO,2400000,0.676,Stressed,4520844650PO1NC5A7S_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,45208_44650_PO1_NC5_A7_1_val_1_bismark_bt2_pe....,2821858,11608,NaN,NaN
4146045001PO2NC3F4C,4146045001PO2NC3F4,41460,45001,f2,2,2023,Pooled Oocytes (3) #2,F4,CTGAACGT,PO,2500000,0.686,Control,4146045001PO2NC3F4C_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,41460_45001_PO2_NC3_F4_1_val_1_bismark_bt2_pe....,2635056,48304,NaN,NaN
4520844650PO2NC3B7S,4520844650PO2NC3B7,45208,44650,n1,1,2022,Pooled Oocytes (3) #2,B7,CTCTGGAT,PO,2600000,0.680,Stressed,4520844650PO2NC3B7S_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,45208_44650_PO2_NC3_B7_1_val_1_bismark_bt2_pe....,2957798,7806,NaN,NaN
3905345005PO1NC2I1S,3905345005PO1NC2I1,39053,45005,b1,1,2023,Pooled Oocytes (2) #1,I1,TGATAGGC,PO,2700000,0.623,Stressed,3905345005PO1NC2I1S_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,39053_45005_PO1_NC2_I1_1_val_1_bismark_bt2_pe....,2784702,20544,NaN,NaN
4129944963PO1NC3N1S,4129944963PO1NC3N1,41299,44963,d2,2,2023,Pooled Oocytes (3) #1,N1,ATTCCGCT,PO,2900000,0.626,Stressed,4129944963PO1NC3N1S_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,41299_44963_PO1_NC3_N1_1_val_1_bismark_bt2_pe....,2278410,121886,NaN,NaN
4211144957PO1NC3K4C,4211144957PO1NC3K4,42111,44957,g1,1,2023,Pooled Oocytes (3) #1,K4,ACTGGTGT,PO,3200000,0.674,Control,4211144957PO1NC3K4C_1_val_1_bismark_bt2_pe.all...,/share/lasallelab/Ensi/project/allcools/allc_P...,42111_44957_PO1_NC3_K4_1_val_1_bismark_bt2_pe....,3039304,77312,NaN,NaN
4660644938PO2NC3A10C,4660644938PO2NC3A1,46606,44938,r1,1,2023,Pooled Oocytes (3) #2,A10,GATCTTGC,PO,3600000,0.680,Control,4660644938PO2NC3A10C_1_val_1_bismark_bt2_pe.al...,/share/lasallelab/Ensi/project/allcools/allc_P...,46606_44938_PO2_NC3_A10_1_val_1_bismark_bt2_pe...,3742728,50728,NaN,NaN


In [7]:
metadata.index


Index(['4146045001PO3NC4G4C', '4129944963PO2NC4M1S', '4520844650PO1NC5A7S',
       '4146045001PO2NC3F4C', '4520844650PO2NC3B7S', '3905345005PO1NC2I1S',
       '4129944963PO1NC3N1S', '4211144957PO1NC3K4C', '4660644938PO2NC3A10C',
       '4139144923PO4NC4K3C', '4598244908PO1NC3J8C', '4598244908PO2NC4K8C',
       '4566944964PO2NC4C7C', '4129944963PO3NC3O1S', '4139144923PO3NC5J3C',
       '4566944964PO3NC3D8C', '4265145042PO3NC4G5S', '4598245001PO2NC4F8C',
       '4499445005PO2NC3C5C', '4660644993PO2NC5D10C', '4566944964PO4NC3E8C',
       '4660644938PO3NC5B10C', '4636344664PO1NC5L9C', '4566944700PO1NC5O8C',
       '4636344914PO1NC4H9C', '4265145042PO2NC4N4S', '4660644993PO3NC5E10C',
       '4211144957PO2NC3L4C', '3889744700PO1NC5D1S', '4265145042PO1NC3E5S',
       '4660644993PO4NC5F10C', '4146044643PO3NC5A4C', '4146045001PO1NC5E4C',
       '4139144650PO1NC4F3C', '4139144650PO2NC4G3C', '4520844650PO3NC5C8S',
       '4566944700PO2NC5P8C', '4146044643PO1NC5O3C', '4456444991PO1NC3H6C',
       

In [8]:
use_features = pd.read_csv(feature_path, header=None, index_col=0).index
use_features.name = var_dim
use_features

Index(['chr1_0', 'chr1_2', 'chr1_3', 'chr1_4', 'chr1_5', 'chr1_6', 'chr1_7',
       'chr1_8', 'chr1_9', 'chr1_10',
       ...
       'chrX_1524', 'chrX_1525', 'chrX_1526', 'chrX_1527', 'chrX_1528',
       'chrX_1529', 'chrX_1530', 'chrX_1531', 'chrX_1532',
       'chrX_NW_021160383v1_random_0'],
      dtype='object', name='chrom100k', length=23803)

In [10]:
#MCDS
total_mcds = MCDS.open(mcds_path,
                       var_dim=var_dim,
                       use_obs=metadata.index).sel({var_dim: use_features})
total_mcds

<xarray.MCDS>
Dimensions:          (cell: 46, chrom100k: 23803, count_type: 2, mc_type: 3)
Coordinates:
  * cell             (cell) <U20 '4146045001PO3NC4G4C' ... '4123744641PO1NC4N2S'
  * chrom100k        (chrom100k) <U29 'chr1_0' ... 'chrX_NW_021160383v1_rando...
    chrom100k_chrom  (chrom100k) <U27 'chr1' ... 'chrX_NW_021160383v1_random'
    chrom100k_end    (chrom100k) int64 100000 300000 400000 ... 153300000 68997
    chrom100k_start  (chrom100k) int64 0 200000 300000 ... 153100000 153200000 0
  * count_type       (count_type) <U3 'mc' 'cov'
  * mc_type          (mc_type) <U3 'CHN' 'CAN' 'CGN'
Data variables:
    chrom100k_da     (cell, chrom100k, mc_type, count_type) uint32 dask.array<chunksize=(26, 6157, 1, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [11]:
#add mC Rate 
total_mcds.add_mc_rate(var_dim=var_dim,
                       normalize_per_cell=True,
                       clip_norm_value=10)

total_mcds

<xarray.MCDS>
Dimensions:            (cell: 46, chrom100k: 23803, count_type: 2, mc_type: 3)
Coordinates:
  * cell               (cell) <U20 '4146045001PO3NC4G4C' ... '4123744641PO1NC...
  * chrom100k          (chrom100k) <U29 'chr1_0' ... 'chrX_NW_021160383v1_ran...
    chrom100k_chrom    (chrom100k) <U27 'chr1' ... 'chrX_NW_021160383v1_random'
    chrom100k_end      (chrom100k) int64 100000 300000 ... 153300000 68997
    chrom100k_start    (chrom100k) int64 0 200000 300000 ... 153200000 0
  * count_type         (count_type) <U3 'mc' 'cov'
  * mc_type            (mc_type) <U3 'CHN' 'CAN' 'CGN'
Data variables:
    chrom100k_da       (cell, chrom100k, mc_type, count_type) uint32 dask.array<chunksize=(26, 6157, 1, 1), meta=np.ndarray>
    chrom100k_da_frac  (cell, chrom100k, mc_type) float64 dask.array<chunksize=(26, 6157, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [12]:
# we are not downsampleing, therefore: 
mcds = total_mcds
mcds

<xarray.MCDS>
Dimensions:            (cell: 46, chrom100k: 23803, count_type: 2, mc_type: 3)
Coordinates:
  * cell               (cell) <U20 '4146045001PO3NC4G4C' ... '4123744641PO1NC...
  * chrom100k          (chrom100k) <U29 'chr1_0' ... 'chrX_NW_021160383v1_ran...
    chrom100k_chrom    (chrom100k) <U27 'chr1' ... 'chrX_NW_021160383v1_random'
    chrom100k_end      (chrom100k) int64 100000 300000 ... 153300000 68997
    chrom100k_start    (chrom100k) int64 0 200000 300000 ... 153200000 0
  * count_type         (count_type) <U3 'mc' 'cov'
  * mc_type            (mc_type) <U3 'CHN' 'CAN' 'CGN'
Data variables:
    chrom100k_da       (cell, chrom100k, mc_type, count_type) uint32 dask.array<chunksize=(26, 6157, 1, 1), meta=np.ndarray>
    chrom100k_da_frac  (cell, chrom100k, mc_type) float64 dask.array<chunksize=(26, 6157, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [13]:
# Highly variable features
#mCH

if hvf_method == 'SVR':
    # use SVR based method
    mch_hvf = mcds.calculate_hvf_svr(var_dim=var_dim,
                                     mc_type=mch_pattern,
                                     n_top_feature=n_top_feature,
                                     plot=True)
else:
    # use bin based method
    mch_hvf = mcds.calculate_hvf(var_dim=var_dim,
                                 mc_type=mch_pattern,
                                 min_mean=0,
                                 max_mean=5,
                                 n_top_feature=n_top_feature,
                                 bin_min_features=5,
                                 mean_binsize=0.05,
                                 cov_binsize=100)

/home/ehabibi/.conda/envs/allcools/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/ehabibi/.conda/envs/allcools/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


n_top_feature is than total number of features, will use all features
Fitting SVR with gamma 0.0420, predicting feature dispersion using mc_frac_mean and cov_mean.
Total Feature Number:     23803
Highly Variable Feature:  23803 (100.0%)


/home/ehabibi/.conda/envs/allcools/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/ehabibi/.conda/envs/allcools/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/ehabibi/.conda/envs/allcools/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/home/ehabibi/.conda/envs/allcools/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [14]:
#save AnnData

total_mcds.coords[f'{var_dim}_{mch_pattern}_feature_select'] = mcds.coords[
    f'{var_dim}_{mch_pattern}_feature_select']

In [15]:
mch_adata = total_mcds.get_adata(mc_type=mch_pattern,
                           var_dim=var_dim,
                           select_hvf=True)

In [16]:
mch_adata.write_h5ad(f'mCH.HVF.h5ad')

mch_adata

AnnData object with n_obs × n_vars = 46 × 23803
    var: 'chrom', 'end', 'start', 'CHN_mean', 'CHN_dispersion', 'CHN_cov', 'CHN_score', 'CHN_feature_select'

In [17]:
# mCG
if hvf_method == 'SVR':
    # use SVR based method
    mcg_hvf = mcds.calculate_hvf_svr(var_dim=var_dim,
                                     mc_type=mcg_pattern,
                                     n_top_feature=n_top_feature,
                                     plot=True)
else:
    # use bin based method
    mcg_hvf = mcds.calculate_hvf(var_dim=var_dim,
                                 mc_type=mcg_pattern,
                                 min_mean=0,
                                 max_mean=5,
                                 n_top_feature=n_top_feature,
                                 bin_min_features=5,
                                 mean_binsize=0.02,
                                 cov_binsize=20)

n_top_feature is than total number of features, will use all features
Fitting SVR with gamma 0.0420, predicting feature dispersion using mc_frac_mean and cov_mean.
Total Feature Number:     23803
Highly Variable Feature:  23803 (100.0%)


In [18]:
# save AnnData
total_mcds.coords[f'{var_dim}_{mcg_pattern}_feature_select'] = mcds.coords[
    f'{var_dim}_{mcg_pattern}_feature_select']

In [19]:
mcg_adata = total_mcds.get_adata(mc_type=mcg_pattern,
                                 var_dim=var_dim,
                                 select_hvf=True)

mcg_adata.write_h5ad(f'mCG.HVF.h5ad')

mcg_adata

AnnData object with n_obs × n_vars = 46 × 23803
    var: 'chrom', 'end', 'start', 'CHN_mean', 'CHN_dispersion', 'CHN_cov', 'CHN_score', 'CHN_feature_select', 'CGN_mean', 'CGN_dispersion', 'CGN_cov', 'CGN_score', 'CGN_feature_select'